In [4]:
!unzip -q capstone1.zip

In [5]:
!unzip -q capstone2.zip

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import os
import shutil
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

#Pretain Layer

In [2]:
# Dataset untuk training
train_dataset = tf.keras.utils.image_dataset_from_directory(
    directory='./DATASET/TRAIN',  # Path ke folder TRAIN
    labels='inferred',                   # Label otomatis dari nama folder
    label_mode='binary',                    # Label numerik (0 atau 1)
    image_size=(120, 120),               # Ukuran gambar (ubah sesuai model Anda)
    batch_size=128                        # Ukuran batch
)

# Dataset untuk testing
test_dataset = tf.keras.utils.image_dataset_from_directory(
    directory='./DATASET/TEST',   # Path ke folder TEST
    labels='inferred',                   # Label otomatis dari nama folder
    label_mode='binary',                    # Label numerik (0 atau 1)
    image_size=(120, 120),               # Ukuran gambar (ubah sesuai model Anda)
    batch_size=128                       # Ukuran batch
)

# Dataset untuk training
train_dataset2 = tf.keras.utils.image_dataset_from_directory(
    directory='./DATASET3/TRAIN',  # Path ke folder TRAIN
    labels='inferred',                   # Label otomatis dari nama folder
    label_mode='binary',                    # Label numerik (0 atau 1)
    image_size=(120, 120),               # Ukuran gambar (ubah sesuai model Anda)
    batch_size=128                        # Ukuran batch
)

# Dataset untuk testing
test_dataset2 = tf.keras.utils.image_dataset_from_directory(
    directory='./DATASET3/TEST',   # Path ke folder TEST
    labels='inferred',                   # Label otomatis dari nama folder
    label_mode='binary',                    # Label numerik (0 atau 1)
    image_size=(120, 120),               # Ukuran gambar (ubah sesuai model Anda)
    batch_size=128                       # Ukuran batch
)

Found 22564 files belonging to 2 classes.
Found 2513 files belonging to 2 classes.
Found 22564 files belonging to 2 classes.
Found 2513 files belonging to 2 classes.


In [3]:
train_dataset = train_dataset.concatenate(train_dataset2)
test_dataset = test_dataset.concatenate(test_dataset2)

In [4]:
# Optimize the datasets for training
SHUFFLE_BUFFER_SIZE = 10000
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE

train_dataset = (train_dataset
                       .cache()
                       .shuffle(SHUFFLE_BUFFER_SIZE)
                       .prefetch(PREFETCH_BUFFER_SIZE)
                       )

test_dataset = (test_dataset
                            .cache()
                            .prefetch(PREFETCH_BUFFER_SIZE)
                            )

In [17]:
# Get the first batch of images and labels
for images, labels in train_dataset.take(5):
	example_batch_images = images
	example_batch_labels = labels
example_batch_labels[0]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-8506159ca601>", line 2, in <cell line: 2>
    for images, labels in train_dataset.take(5):
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 826, in __next__
    return self._next_internal()
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 776, in _next_internal
    ret = gen_dataset_ops.iterator_get_next(
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 3081, in iterator_get_next
    _result = pywrap_tfe.TFE_Py_FastPathExecute(
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.p

TypeError: object of type 'NoneType' has no len()

In [5]:
augmentation_model = tf.keras.Sequential([
# Specify the input shape.
    tf.keras.Input(shape=(120, 120, 3)),
    # Add the augmentation layers
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2, fill_mode='nearest'),
    tf.keras.layers.RandomTranslation(0.2,0.2, fill_mode='nearest'),
    tf.keras.layers.RandomZoom(0.2, fill_mode='nearest')])

In [ ]:
# Load your model for augmentation
data_augmentor = augmentation_model

# Take a sample image
sample_image = tf.keras.utils.array_to_img(example_batch_images[0])

images = [sample_image]

# Apply random augmentation 3 times
for _ in range(3):
	image_aug = data_augmentor(tf.expand_dims(sample_image, axis=0))
	image_aug = tf.keras.utils.array_to_img(tf.squeeze(image_aug))
	images.append(image_aug)


fig, axs = plt.subplots(1, 4, figsize=(14, 7))
for ax, image, title in zip(axs, images, ['Original Image', 'Augmented 1', 'Augmented 2', 'Augmented 3']):
    ax.imshow(image, cmap='gray')
    ax.set_title(title)
    ax.axis('off')

plt.show()

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(120, 120, 3)),
    augmentation_model,
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')

])

# Set the training parameters
model.compile(loss = 'sparse_categorical_crossentropy',  optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [19]:
try:
	model.evaluate(example_batch_images, example_batch_labels, verbose=False)
except:
	print("Your model is not compatible with the dataset you defined earlier. Check that the loss function, last layer and label_mode are compatible with one another.")
else:
	predictions = model.predict(example_batch_images, verbose=False)
	print(f"predictions have shape: {predictions.shape}")

Your model is not compatible with the dataset you defined earlier. Check that the loss function, last layer and label_mode are compatible with one another.


In [28]:
# GRADED CLASS: EarlyStoppingCallback

### START CODE HERE ###

# Remember to inherit from the correct class
class EarlyStoppingCallback(tf.keras.callbacks.Callback):
    # Define the correct function signature for on_epoch_end method
    def on_epoch_end(self, epoch, logs={}):
        # Check if the accuracy is greater or equal to 0.8 and validation accuracy is greater or equal to 0.8
        if logs['accuracy'] >= 0.8 and logs['val_accuracy'] >= 0.85:
            self.model.stop_training = True

            print("\nReached 80% train accuracy and 80% validation accuracy, so cancelling training!")

### END CODE HERE ###

In [7]:
# Train the model and save the training history
# This may take up to 10-15 min so feel free to take a break! :P
history = model.fit(
	train_dataset,
	epochs=10,
	validation_data=test_dataset,
)

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
# Get training and validation accuracies
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle('Training and validation accuracy')

for i, (data, label) in enumerate(zip([(acc, val_acc), (loss, val_loss)], ["Accuracy", "Loss"])):
    ax[i].plot(epochs, data[0], 'r', label="Training " + label)
    ax[i].plot(epochs, data[1], 'b', label="Validation " + label)
    ax[i].legend()
    ax[i].set_xlabel('epochs')

plt.show()

In [ ]:
import tensorflow as tf
import numpy as np

# Fungsi untuk memuat dan memproses gambar
def load_and_preprocess_image(image_path, img_size=(120, 120)):
    # Muat gambar
    img = tf.keras.utils.load_img(image_path, target_size=img_size)  # Load gambar dan ubah ukurannya
    # Konversi ke array
    img_array = tf.keras.utils.img_to_array(img)
    # Tambahkan dimensi batch
    img_array = np.expand_dims(img_array, axis=0)
    # Normalisasi (skala 0–1)
    img_array = img_array / 255.0
    return img_array

In [ ]:
# Path ke gambar baru
image_path = 'botol.jpg'  # Ganti dengan path gambar Anda

# Preproses gambar
processed_image = load_and_preprocess_image(image_path)
# Lakukan prediksi
predictions = model(processed_image)
predicted_class = np.argmax(predictions[0])  # Ambil indeks kelas dengan probabilitas tertinggi
print(f"Predicted Class: {predicted_class}")

#Functional API

In [ ]:
## Please uncomment all lines in this cell and replace those marked with `# YOUR CODE HERE`.
## You can select all lines in this code cell with Ctrl+A (Windows/Linux) or Cmd+A (Mac), then press Ctrl+/ (Windows/Linux) or Cmd+/ (Mac) to uncomment.
class Block(tf.keras.Model):
    def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
        super(Block, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions

        # Define a conv2D_0, conv2D_1, etc based on the number of repetitions
        for i in range(self.repetitions):

            # Define a Conv2D layer, specifying filters, kernel_size, activation and padding.
            vars(self)[f'conv2D_{i}'] =  tf.keras.layers.Conv2D(
                    filters=self.filters,
                    kernel_size=self.kernel_size,
                    activation="relu",  # Assuming ReLU activation
                    padding="same"
                )

        # Define the max pool layer that will be added after the Conv2D blocks
        self.max_pool = tf.keras.layers.MaxPool2D(pool_size=pool_size, strides=strides)

    def call(self, inputs):
        # access the class's conv2D_0 layer
        conv2D_0 = vars(self)[f'conv2D_0']

        # Connect the conv2D_0 layer to inputs
        x = conv2D_0(inputs)

        # for the remaining conv2D_i layers from 1 to `repetitions` they will be connected to the previous layer
        for i in range(1, self.repetitions):
            # access conv2D_i by formatting the integer `i`. (hint: check how these were saved using `vars()` earlier)
            conv2D_i = vars(self)[f'conv2D_{i}']

            # Use the conv2D_i and connect it to the previous layer
            x = conv2D_i(x)

        # Finally, add the max_pool layer
        max_pool = self.max_pool(x)

        return max_pool

In [ ]:
## Please uncomment all lines in this cell and replace those marked with `# YOUR CODE HERE`.
## You can select all lines in this code cell with Ctrl+A (Windows/Linux) or Cmd+A (Mac), then press Ctrl+/ (Windows/Linux) or Cmd+/ (Mac) to uncomment.

class MyVGG(tf.keras.Model):

    def __init__(self, num_classes):
        super(MyVGG, self).__init__()

        # Creating blocks of VGG with the following
        # (filters, kernel_size, repetitions) configurations
        self.block_a = Block(filters=64, kernel_size=3, repetitions=2)
        self.block_b = Block(filters=128, kernel_size=3, repetitions=2)
        self.block_c = Block(filters=256, kernel_size=3, repetitions=3)
        self.block_d = Block(filters=512, kernel_size=3, repetitions=3)
        self.block_e = Block(filters=512, kernel_size=3, repetitions=3)

        # Classification head
        # Define a Flatten layer
        self.flatten = tf.keras.layers.Flatten()
        # Create a Dense layer with 256 units and ReLU as the activation function
        self.fc = tf.keras.layers.Dense(units=256, activation='relu')
        # Finally add the softmax classifier using a Dense layer
        self.classifier = tf.keras.layers.Dense(units=2, activation='softmax')

    def call(self, inputs):
        # Chain all the layers one after the other
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.classifier(x)
        return x

In [ ]:
# For reference only. Please do not uncomment in Coursera Labs because it might cause the grader to time out.
# You can upload your notebook to Colab instead if you want to try the code below.

# Initialize VGG with the number of classes
vgg = MyVGG(num_classes=2)

vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#
# Define preprocessing function
def preprocess(features):
    # Resize and normalize
    image = tf.image.resize(features['image'], (120, 120))
    return tf.cast(image, tf.float32) / 255., features['label']


# Train the custom VGG model
history = vgg.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=5,
)

In [ ]:
# Evaluasi model
loss, accuracy = vgg.evaluate(test_dataset)
print(f"Loss: {loss}, Accuracy: {accuracy}")

In [ ]:
# Path ke gambar baru
image_path = 'botol.jpg'  # Ganti dengan path gambar Anda

# Preproses gambar
processed_image = load_and_preprocess_image(image_path)
# Lakukan prediksi
predictions = vgg.predict(processed_image)
predicted_class = np.argmax(predictions[1])  # Ambil indeks kelas dengan probabilitas tertinggi
print(f"Predicted Class: {predicted_class}")

#Kaggle

In [ ]:
transforms = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
train_dataset = ImageFolder(root='./DATASET/TRAIN', transform=transforms)
test_dataset = ImageFolder(root='./DATASET/TEST', transform=transforms)

In [ ]:
train_size = int(0.8*len(train_dataset))
val_size = len(train_dataset) - train_size

train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
len(train_dataset), len(val_dataset)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=2)

len(train_loader), len(val_loader), len(test_loader)

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.model = nn.Sequential(
            # first layer
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),

            # second layer
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),

            # third layer
            nn.Flatten(),
            nn.Linear(in_features=29*29*16, out_features=120),
            nn.ReLU(),

            # fourth layer
            nn.Linear(in_features=120, out_features=84),
            nn.ReLU(),

            # output layer
            nn.Linear(in_features=84, out_features=2)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
model = LeNet()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
epochs = 5

training_loss = []
validation_loss = []

for epoch in range(epochs):

    # Training phase
    model.train()
    epoch_train_loss = 0.0
    for images, labels in train_loader:
        # Move data to the appropriate device

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        train_loss = loss_fn(outputs, labels)

        # Backward pass
        train_loss.backward()
        optimizer.step()

        epoch_train_loss += train_loss.item() * images.size(0)  # Accumulate loss

    # Average training loss for the epoch
    epoch_train_loss /= len(train_loader.dataset)
    training_loss.append(epoch_train_loss)
    model.eval()
    epoch_val_loss = 0.0
    with torch.inference_mode():
        for val_images, val_labels in val_loader:
            # Move validation data to the appropriate device

            val_outputs = model(val_images)
            val_loss = loss_fn(val_outputs, val_labels)
            epoch_val_loss += val_loss.item() * val_images.size(0)  # Accumulate loss

    # Average validation loss for the epoch
    epoch_val_loss /= len(val_loader.dataset)
    validation_loss.append(epoch_val_loss)

    # Print loss for the epoch
    print(f"Epoch: {epoch+1}/{epochs}, Training Loss: {epoch_train_loss:.4f}, Validation Loss: {epoch_val_loss:.4f}")

In [ ]:
predicted_labels = []
actual_labels = []

model.eval()
with torch.inference_mode():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predicted_labels.extend(predicted.cpu().numpy())
        actual_labels.extend(labels.cpu().numpy())

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report

accuracy = accuracy_score(actual_labels, predicted_labels)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# print precision

precision = precision_score(actual_labels, predicted_labels, average='weighted')
print(f"Model Precision: {precision * 100:.2f}%")

# print recall
recall = recall_score(actual_labels, predicted_labels, average='weighted')
print(f"Model Recall: {recall * 100:.2f}%")

# print f1 score

f1 = f1_score(actual_labels, predicted_labels, average='weighted')
print(f"Model F1 Score: {f1 * 100:.2f}%")

# print classification report

print(classification_report(actual_labels, predicted_labels))
# get confsion matrix and print actual labels

cm = confusion_matrix(actual_labels, predicted_labels)

class_names = test_loader.dataset.classes
# visualize confusion matruix using sns
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()